In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pymongo import MongoClient
import json

In [13]:
# User Imput

print("Please imput a zip code.")
zip = input()
zzip = int(zip)

Please imput a zip code.
99109


In [14]:
# Getting Lat/Long from github data source

url = "https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data"

df = pd.read_csv(url)
data = df[df['ZIP']== zzip]

# if dataframe empty, stop the program here.

latitude = data['LAT'].values[0]
lng = data['LNG'].values[0]

In [15]:
# Getting URL from Lat/Long

str = "lat={lat}&lon={lon}".format(lat = latitude, lon = lng)
url2 = "https://forecast.weather.gov/MapClick.php?{location}".format(location = str)
print(url2)

https://forecast.weather.gov/MapClick.php?lat=48.292946&lon=-117.698546


In [16]:
#Getting Data for 7 day table

page = requests.get(url2)
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")

period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]


In [17]:
# Getting name of location

current_conditions = soup.find(id="current-conditions")
local = current_conditions.select(".panel-heading .panel-title")
city = [pt.get_text() for pt in local]
cityname = city[0]

In [18]:
# 7 day weather forcast

print("7 day forcast for", cityname)
weather = pd.DataFrame({
 "period": periods,
 "short_desc": short_descs,
 "temp": temps,
 "desc":descs
})
weather

7 day forcast for PAL MOORE ORCHARD (PMFW1)


,period,short_desc,temp,desc
0,Today,Chance Snow,High: 19 °F,Today: A 30 percent chance of snow. Mostly cl...
1,Tonight,Heavy Snow,Low: 16 °F,"Tonight: Snow, mainly after 10pm. The snow cou..."
2,Wednesday,Heavy Snow,High: 29 °F,Wednesday: Snow. The snow could be heavy at ti...
3,WednesdayNight,Snow andPatchy Fog,Low: 20 °F,"Wednesday Night: Snow. Patchy fog before 2am,..."
4,Thursday,Chance Snow,High: 30 °F,Thursday: A 40 percent chance of snow. Cloudy...
5,ThursdayNight,Slight ChanceSnow,Low: 10 °F,Thursday Night: A 20 percent chance of snow. ...
6,Friday,Slight ChanceSnow,High: 22 °F,Friday: A 20 percent chance of snow. Mostly c...
7,FridayNight,Chance Snow,Low: 14 °F,Friday Night: A 40 percent chance of snow. Cl...
8,Saturday,Chance Snow,High: 26 °F,"Saturday: A 30 percent chance of snow, mainly ..."


In [26]:
# Current Weather

table = current_conditions.select("table")
rows = soup.find_all('td')
rnames = [td.get_text() for td in rows]
print("Current Weather in", cityname)

time = rnames[11]
time2 = time.strip()

h = rnames[1]
w = rnames[3]
d = rnames[7]



current = pd.DataFrame({
    "Humidity": h,
    "Wind Speed": w,
    "Dewpoint": d,
    "Updated at": time2
    
}, index = [0])

current

Current Weather in PAL MOORE ORCHARD (PMFW1)


,Humidity,Wind Speed,Dewpoint,Updated at
0,88%,NA 0 MPH,12°F (-11°C),29 Nov 08:55 AM PST


In [46]:
# Inserting into Mongo
# pip install pymongo




cluster = MongoClient("mongodb+srv://cluster0.fc5vawu.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority")
db = cluster["lab5"]
collection1 = db["7dayforecast"]
collection2 = db["currentconditions"]


sevenday = weather.to_dict("records")
cc = current.to_dict("records")

#collection1.insert_many(sevenday)


[{'Humidity': '88%',
  'Wind Speed': 'NA 0 MPH',
  'Dewpoint': '12°F (-11°C)',
  'Updated at': '29 Nov 08:55 AM PST'}]